# 1. Import thư viện và nạp dữ liệu vào notebook

In [23]:
#Importing the Necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
data = pd.read_csv('mushrooms.csv', encoding='latin-1')
#display the first 5 rows
data.head()


,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


# 2. Xử lý dữ liệu trước khi xây dựng mô hình từ dữ liệu

In [24]:
X=data.drop('class', axis=1)
y=data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
random_state=42)

# 3. Xây dựng vector hóa nội dung của tập train và tập test

In [25]:
from sklearn.compose import ColumnTransformer
# Xác định các cột cần mã hóa (tất cả các cột trong X)
categorical_features = X.columns.tolist() 

# 1. Bộ chuyển đổi: Mã hóa One-Hot
categorical_transformer = OneHotEncoder(
    handle_unknown='ignore', 
    sparse_output=False # Quan trọng để GaussianNB có thể xử lý
)

# 2. ColumnTransformer (Áp dụng One-Hot cho tất cả các cột)
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Mặc dù không còn cột nào, vẫn nên đặt.
)

# --- Mã hóa Nhãn Y (Label Encoding) ---
# GaussianNB (và nhiều mô hình khác) làm việc tốt hơn nếu nhãn y là số (0 hoặc 1)
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# 4. Xây dựng mô hình Naive Bayes

In [26]:
from sklearn.pipeline import Pipeline
# 3. Tạo Pipeline hoàn chỉnh
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GaussianNB()) # Sử dụng GaussianNB
])

# 4. Huấn luyện mô hình
# Pipeline sẽ tự động: One-Hot Encoder X_train -> Huấn luyện GaussianNB
print("Đang huấn luyện mô hình GaussianNB...")
model.fit(X_train, y_train_encoded) 
print("Huấn luyện hoàn tất!")

Đang huấn luyện mô hình GaussianNB...
Huấn luyện hoàn tất!


# 5. Đánh giá hiệu quả của mô hình

In [27]:
from sklearn.metrics import accuracy_score, classification_report

# 1. Dự đoán trên tập kiểm thử
# Pipeline sẽ tự động: One-Hot Encoder X_test -> Dự đoán
y_pred_encoded = model.predict(X_test)

# 2. Tính Độ chính xác (Accuracy)
accuracy = accuracy_score(y_test_encoded, y_pred_encoded)

# 3. Báo cáo chi tiết (Giải mã nhãn 0/1 thành e/p)
target_names = le.classes_ # Lấy lại tên nhãn gốc ('e', 'p')

print("\n--- KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH ---")
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report(y_test_encoded, y_pred_encoded, target_names=target_names))


--- KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH ---
Accuracy: 0.9600
Classification Report:
              precision    recall  f1-score   support

           e       1.00      0.92      0.96       843
           p       0.92      1.00      0.96       782

    accuracy                           0.96      1625
   macro avg       0.96      0.96      0.96      1625
weighted avg       0.96      0.96      0.96      1625

